In [ ]:
import boto3
import zipfile
import shutil
import os
import glob
from distutils.dir_util import copy_tree

In [ ]:
# aws_access_key_id = ''
# aws_secret_access_key = ''
# aws_session_token = ''

s3 = boto3.client('s3', aws_access_key_id=aws_access_key_id,
                  aws_secret_access_key=aws_secret_access_key,
                  aws_session_token=aws_session_token)

In [ ]:
output_path = '../deep/'

# paths = open('bucket_list.txt', 'r')
paths = open('dashboard.txt', 'r')
Lines = paths.readlines()


# for line in Lines[0:1]:
for k in range(len(Lines)):
# for k in range(1):
    
    line = Lines[k]

    path = os.path.dirname(line[:-1])
    response = s3.list_objects_v2(Bucket='validacao-retrospectiva-neuro-new', Prefix=path)

    image_path = line[:-1]
    print("Line :",k,  image_path)
    final_path = output_path + os.path.basename(image_path)
    
    print(final_path[8:-40])
    try:
        s3.download_file('validacao-retrospectiva-neuro-new', image_path, final_path)
        
        with zipfile.ZipFile(final_path, 'r') as zip_ref:
            zip_ref.extractall(output_path)      
        os.remove(final_path)
        
        
        file_from = glob.glob('../deep/nii_images_series_*/gaussian_image.nii.gz')[0]
        file_to = '../img/' + final_path[8:-40] + '/' + final_path[8:-40] + '_gaussian_image.nii.gz'
        
        os.mkdir(os.path.dirname(file_to))   
        shutil.move(file_from, file_to)
        
              
        file_from = glob.glob('../deep/nii_images_series_*/master_image.nii.gz')[0]
        file_to = output_path + final_path[8:-40] + '/' + final_path[8:-40] + '_master_image.nii.gz'
        
        os.mkdir(os.path.dirname(file_to))   
        shutil.move(file_from, file_to)

        
        
        file_from = glob.glob('../deep/nii_images_series_*/brain_label.nii.gz')[0]
        file_to = output_path + final_path[8:-40] + '/' + final_path[8:-40] + '_brain_label.nii.gz'
        
        shutil.move(file_from, file_to)
        
        
        file_from = glob.glob('../deep/nii_images_series_*/ventricle_label.nii.gz')[0]
        file_to = output_path + final_path[8:-40] + '/' + final_path[8:-40] + '_ventricle_label.nii.gz'
        
        shutil.move(file_from, file_to)
        
        
        file_from = glob.glob('../deep/nii_images_series_*/brain_g_truth.nii.gz')[0]
        file_to = output_path + final_path[8:-40] + '/' + final_path[8:-40] + '_brain_truth.nii.gz'
        
        shutil.move(file_from, file_to)
        
        
        
        shutil.rmtree( os.path.dirname(file_from) )
        
        # os.rename(final_path[:-4], final_path[:-40])
        
        print("Finished")
    except:
        pass

In [ ]:
exam_path = "/volumed/Anonymized_DICOMs/Anonymized_DICOMs/"

exam_list = glob.glob(exam_path + 'id_*/nii_images_series*/')

bucket_name = 'validacao-retrospectiva-neuro-new'

for exam in sorted(exam_list)[3:24]:
    
    try:
        
        folder_name = exam[:-1]
        series_name = os.path.basename(folder_name)
        id_name = os.path.basename(os.path.dirname(folder_name))
        
        folder_from = folder_name
        folder_to = os.path.join(os.path.dirname(folder_name), id_name, series_name)
        
        copy_tree(folder_from, folder_to)
        
    
        shutil.make_archive(os.path.dirname(folder_to), 'zip', os.path.dirname(folder_to))
        
        # Filename - File to upload
        # Bucket - Bucket to upload to (the top level directory under AWS S3)
        # Key - S3 object name (can contain subdirectories). If not specified then file_name is used
        # s3.meta.client.upload_file(Filename='input_file_path', Bucket='bucket_name', Key='s3_output_key')
        s3.upload_file(os.path.dirname(folder_to) + '.zip', bucket_name, 'dashboard_base_retrospectiva/' + id_name + '.zip')
        
        shutil.rmtree(os.path.dirname(folder_to))
        os.remove(os.path.dirname(folder_to) + '.zip')
        
        print(id_name)
        print(os.path.dirname(folder_to))
        
    except:
        pass

In [ ]:
pip install boto3